## 最尤推定

In [2]:
// 依存関係のインストール
:dep image = "0.23"
:dep evcxr_image = "1.1"

:dep rand = "0.8.5"

// プロット用ライブラリ
:dep plotters = { version = "^0.3.5", default_features = false, features = ["evcxr", "all_series", "all_elements"] }

In [3]:
// インポート
use evcxr_image::ImageDisplay;
use image::{GenericImage, imageops::FilterType};

use plotters::prelude::*;
use rand::prelude::*;

In [4]:
use std::{fs::File, io::{self, prelude::*}, path::Path};

In [5]:
// 自作ライブラリ
:dep myml = { path = "../myml/" }

In [6]:
use myml::{utility::linspace, normal::normal};

### データの読み込み

In [7]:
// ファイル
let height_file_path = Path::new("./data/height.txt");

// 読み込み
let height_file = File::open(&height_file_path)?;
let reader = io::BufReader::new(height_file);
let height_data = reader
    .lines()
    .filter_map(|line| line.ok())
    .filter_map(|val| val.parse::<f64>().ok())
    .collect::<Vec<_>>();

// 読み取れていることを確認
&height_data[..5]

[167.089607, 181.648633, 176.2728, 173.270164, 172.181037]

In [8]:
let count = height_data.len();

count

25000

### ヒストグラムをプロット

In [9]:
evcxr_figure((640, 480), |root| {
    root.fill(&WHITE)?;

    let mut chart = ChartBuilder::on(&root)
        .caption("height", ("Roman", 20).into_font())
        .x_label_area_size(50)
        .y_label_area_size(50)
        .build_cartesian_2d(150..200, 0.0..0.1)?;

    chart.configure_mesh()
        .draw()?;

    // ヒストグラムを描画
    let hist = Histogram::vertical(&chart)
        .style(BLUE.filled())
        .margin(0)
        .data(height_data.iter().map(|&x| (x as i32, 1.0 / count as f64)));

    chart.draw_series(hist)?;

    Ok(())
})

height
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0.0
 
 
 
0.01
 
 
 
0.02
 
 
 
0.03
 
 
 
0.04
 
 
 
0.05
 
 
 
0.06
 
 
 
0.07
 
 
 
0.08
 
 
 
0.09
 
 
 
0.1
 
 
 
 
150
 
 
 
155
 
 
 
160
 
 
 
165
 
 
 
170
 
 
 
175
 
 
 
180
 
 
 
185
 
 
 
190
 
 
 
195
 
 
 
200

### 正規分布による生成モデル

#### やること

- モデル化：身長データが正規分布に従うと仮定
- パラメータ推定：モデル（ここでは正規分布）のパラメータを推定する

すなわち，

$$
f(x; \mu, \sigma) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left( -\frac{(x - \mu)^2}{2\sigma^2} \right)
$$

の $\mu,\sigma$ を推定する．

In [10]:
// 身長の平均
let height_mean = height_data.iter().sum::<f64>() / count as f64;

height_mean

172.70250853668063

In [11]:
// 身長の分散
let height_var = height_data
    .iter()
    .map(|h| (h - height_mean).powf(2.0))
    .sum::<f64>() / count as f64;

height_var

23.330517821055636

In [12]:
// 身長の標準偏差
let height_std = height_var.sqrt();

height_std

4.830167473396304

In [13]:
// 対応する正規分布を生成
let x = linspace(150.0, 190.0, 1000);
let y = normal(&x, height_mean, height_std);

let points = x.iter().copied().zip(y.iter().copied()).collect::<Vec<_>>();

### プロット

（値が丸められているが，概ね一致していることがわかる）

In [14]:
evcxr_figure((640, 480), |root| {
    root.fill(&WHITE)?;

    let mut chart = ChartBuilder::on(&root)
        .caption("height", ("Roman", 20).into_font())
        .x_label_area_size(50)
        .y_label_area_size(50)
        .build_cartesian_2d(150..200, 0.0..0.1)?;

    chart.configure_mesh()
        .draw()?;

    // ヒストグラムを描画
    let hist = Histogram::vertical(&chart)
        .style(BLUE.filled())
        .margin(0)
        .data(height_data.iter().map(|&x| (x as i32, 1.0 / count as f64)));

    chart.draw_series(hist)?;

    // モデルを描画（同様にxを10倍する）
    chart.draw_series(
        LineSeries::new(
            points.iter().map(|&(x, y)| (x as i32, y)),
            &RED,
        )
    )?;
    
    Ok(())
})

height
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0.0
 
 
 
0.01
 
 
 
0.02
 
 
 
0.03
 
 
 
0.04
 
 
 
0.05
 
 
 
0.06
 
 
 
0.07
 
 
 
0.08
 
 
 
0.09
 
 
 
0.1
 
 
 
 
150
 
 
 
155
 
 
 
160
 
 
 
165
 
 
 
170
 
 
 
175
 
 
 
180
 
 
 
185
 
 
 
190
 
 
 
195
 
 
 
200
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="50,429 50,429 50,429 50,429 50,429 50,429 50,429 50,429 50,429 50,429 50,429 50,429 50,429 50,429 50,429 50,429 50,429 50,429 50,429 50,429 50,429 50,429 50,429 50,429 50,429 61,429 61,429 61,429 61,429 61,429 61,429 61,429 61,429 61,429 61,429 61,429 61,429 61,429 61,429 61,429 61,429 61,429 61,429 61,429 61,429 61,429 61,429 61,429 61,429 61,429 73,429 73,429 73,429 73,429 73,429 73,429 73,429 73,429 73,429 73,429 73,429 73,429 73,429 73,429 73,429 73,429 73,429 73,429 73,429 73,429 73,429 73,429 73,429 73,429 73,429 85,429 85,429 85,429 85,429 85,429 85,429 85,429 85,429 85,429 85,429 85,429 85,429 85,429 85,429 85,429 85,429 85,429 85,429 85,429 85,429 85,429 85,429 85,429 85,429 85,429 97,429 97,429 97,429 97,429 97,429 97,429 97,429 97,429 97,429 97,429 97,429 97,429 97,429 97,429 97,429 97,429 97,429 97,429 97,429 97,429 97,429 97,429 97,429 97,429 97,429 108,429 108,429 108,429 108,429 108,429 108,429 108,429 108,429 108,429 108,429 108,429 108,429 108,429 108,429 108,429 108,429 108,429 108,429 108,429 108,429 108,429 108,429 108,429 108,429 108,429 120,429 120,429 120,429 120,429 120,429 120,429 120,428 120,428 120,428 120,428 120,428 120,428 120,428 120,428 120,428 120,428 120,428 120,428 120,428 120,428 120,428 120,428 120,428 120,428 120,428 132,428 132,428 132,428 132,428 132,428 132,428 132,428 132,427 132,427 132,427 132,427 132,427 132,427 132,427 132,427 132,427 132,427 132,427 132,427 132,427 132,427 132,427 132,426 132,426 132,426 144,426 144,426 144,426 144,426 144,426 144,426 144,426 144,426 144,426 144,425 144,425 144,425 144,425 144,425 144,425 144,425 144,425 144,425 144,424 144,424 144,424 144,424 144,424 144,424 144,424 156,424 156,423 156,423 156,423 156,423 156,423 156,423 156,422 156,422 156,422 156,422 156,422 156,422 156,421 156,421 156,421 156,421 156,421 156,420 156,420 156,420 156,420 156,420 156,419 156,419 167,419 167,419 167,418 167,418 167,418 167,418 167,418 167,417 167,417 167,417 167,416 167,416 167,416 167,416 167,415 167,415 167,415 167,414 167,414 167,414 167,413 167,413 167,413 167,412 167,412 179,412 179,411 179,411 179,411 179,410 179,410 179,409 179,409 179,409 179,408 179,408 179,407 179,407 179,407 179,406 179,406 179,405 179,405 179,404 179,404 179,403 179,403 179,402 179,402 179,401 191,401 191,400 191,400 191,399 191,399 191,398 191,397 191,397 191,396 191,396 191,395 191,394 191,394 191,393 191,393 191,392 191,391 191,391 191,390 191,389 191,389 191,388 191,387 191,386 191,386 203,385 203,384 203,383 203,383 203,382 203,381 203,380 203,380 203,379 203,378 203,377 203,376 203,375 203,375 203,374 203,373 203,372 203,371 203,370 203,369 203,368 203,367 203,366 203,365 203,364 214,363 214,362 214,361 214,360 214,359 214,358 214,357 214,356 214,355 214,354 214,353 214,352 214,351 214,350 214,349 214,348 214,346 214,345 214,344 214,343 214,342 214,341 214,339 214,338 214,337 226,336 226,334 226,333 226,332 226,331 226,329 226,328 226,327 226,325 226,324 226,323 226,321 226,320 226,319 226,317 226,316 226,315 226,313 226,312 226,310 226,309 226,308 226,306 226,305 226,303 238,302 238,300 238,299 238,297 238,296 238,294 238,293 238,291 238,290 238,288 238,287 238,285 238,284 238,282 238,281 238,279 238,277 238,276 238,274 238,273 238,271 238,269 238,268

## データの生成

推定したモデルをもとに，元のデータに従う新たなデータを生成する．

In [15]:
// 乱数
:dep rand_distr = "0.4.3"

In [16]:
use rand::prelude::{thread_rng, Distribution};
use rand_distr::Normal;

In [17]:
// 乱数生成器
let mut rng = thread_rng();

// 分布
let dist = Normal::<f64>::new(height_mean, height_std)?;

In [18]:
// 試しにサンプリング
dist.sample(&mut rng)

176.0373531958946

In [19]:
// 10000個のサンプリング
let sample_size = 10000;
let generated_heights = dist.sample_iter(&mut rng).take(sample_size).collect::<Vec<_>>();

&generated_heights[..5]

[174.296516478635, 173.45843230673194, 181.0376202621591, 174.63869533511337, 173.8480648420886]

元データに重ねてプロットしてみる

In [20]:
evcxr_figure((640, 480), |root| {
    root.fill(&WHITE)?;

    let mut chart = ChartBuilder::on(&root)
        .caption("height data / generated data", ("Roman", 20).into_font())
        .x_label_area_size(50)
        .y_label_area_size(50)
        .build_cartesian_2d(150..200, 0.0..0.1)?;

    chart.configure_mesh()
        .draw()?;

    // 元データのヒストグラムを描画
    let hist = Histogram::vertical(&chart)
        .style(BLUE.filled())
        .margin(0)
        .data(height_data.iter().map(|&x| (x as i32, 1.0 / count as f64)));

    chart.draw_series(hist)?;

    // 生成されたデータのヒストグラムを描画
    let hist2 = Histogram::vertical(&chart)
        .style(RGBAColor(255, 128, 0, 0.4).filled())
        .margin(0)
        .data(generated_heights.iter().map(|&x| (x as i32, 1.0 / sample_size as f64)));

    chart.draw_series(hist2)?;

    Ok(())
})

height data / generated data
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0.0
 
 
 
0.01
 
 
 
0.02
 
 
 
0.03
 
 
 
0.04
 
 
 
0.05
 
 
 
0.06
 
 
 
0.07
 
 
 
0.08
 
 
 
0.09
 
 
 
0.1
 
 
 
 
150
 
 
 
155
 
 
 
160
 
 
 
165
 
 
 
170
 
 
 
175
 
 
 
180
 
 
 
185
 
 
 
190
 
 
 
195
 
 
 
200